In [ ]:
from visualize import *
import numpy as np
import pyBigWig
import torch
import os
import sys

In [ ]:
def _read_hic_matrix(hic_file, chrom, start, end, res,log1p=True, normalization = 'NONE'):
        """
        Reads a Hi-C interaction matrix for a specified genomic region from a Hi-C file.
        Args:
            hic_file (str): Path to the Hi-C file.
            chrom (str): Chromosome name.
            start (int): Start position of the genomic region.
            end (int): End position of the genomic region.
        Returns:
            csr_matrix: A sparse matrix representing the Hi-C interaction matrix for the specified region.
        The function uses the hicstraw library to extract observed interaction values for the specified region.
        It constructs a sparse matrix from these values, ensuring that the matrix is symmetric and the diagonal
        values are halved. If an error occurs during the process, an empty sparse matrix of the appropriate size
        is returned.
        """
        try:
            # Construct the Hi-C location string
            chrom = chrom.replace('chr', '')
            loc = f'{chrom}:{start}:{end}'
            # Extract Hi-C interaction values
            result = hicstraw.straw('observed', normalization, hic_file, loc, loc, 'BP', res)
            
            num_bins = (end - start) // res
            # Use lil_matrix for more efficient sparse matrix construction
            hic_matrix = np.zeros((num_bins, num_bins), dtype=np.float32)
            
            for entry in result:
                bin_x = (entry.binX - start) // res
                bin_y = (entry.binY - start) // res
                
                if 0 <= bin_x < num_bins and 0 <= bin_y < num_bins:
                    hic_matrix[bin_x, bin_y] = entry.counts
                    hic_matrix[bin_y, bin_x] = entry.counts
            
            
            if log1p:
                hic_matrix = np.log1p(hic_matrix)
            return hic_matrix
        except Exception as e:
            print(f'Error reading Hi-C matrix for {chrom}:{start}-{end}: {str(e)}')
            num_bins = (end - start) // res
            return np.zeros((num_bins, num_bins), dtype=np.float32)
            

In [ ]:
rootpath = os.path.dirname(os.path.abspath(__file__).join('/..').join('/..').join('/..'))

# CTCF ChIAPET

In [ ]:
outputpath = f'{rootpath}/gChIA/src/gchia/Figure/2/CTCF'

## 10kb

In [ ]:
cellline = 'GM12878'
# load data
hic = np.load(f'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions/pred_matrix_chr10_21000000_23000000.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/GM12878/bigWig_files/GM12878_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 23000000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/CTCF_ChIA-PET/ENCFF379AWZ.hic'
rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 23000000, 10000,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', 21000000, 23000000, 10000,log1p=False)
rhic = np.nan_to_num(rhic)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['Predicted 10kb','CTCF ChIA-PET 10kb'],epi_labels=['CTCF'],x_ticks=['21MB','chr10','23MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_10kb_ctcf.png')


## 5kb

In [ ]:
# load data
hic = np.load(f'{rootpath}/gChIA/results/250304_5000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_True/GM12878/predictions/pred_matrix_chr10_21000000_23000000.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/GM12878/bigWig_files/GM12878_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 23000000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/CTCF_ChIA-PET/ENCFF379AWZ.hic'
rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 23000000, 5000,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', 21000000, 23000000, 5000,log1p=False)
rhic = np.nan_to_num(rhic)


In [ ]:
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['Predicted 5kb','CTCF ChIA-PET 5kb'],epi_labels=['CTCF'],x_ticks=['21MB','chr10','23MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_5kb_ctcf.png')

## 1kb

In [ ]:
# load data
hic = np.load(f'{rootpath}/gChIA/results/250331_GM12878_1000_NONE_ChIAPETMatrixPredictor_efeaturesq_high_CTCF_ChIA-PET_log1p_true_hictype_hic_chiatype_hic/GM12878/predictions/pred_matrix_chr10_21000000_21500000.npy')
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/GM12878/bigWig_files/GM12878_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 21500000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/CTCF_ChIA-PET/ENCFF379AWZ.hic'
rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 21500000, 1000,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', 21000000, 21500000, 1000,log1p=False)
rhic = np.nan_to_num(rhic)

In [ ]:
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['rChIA 1kb','chr10','ChIA-PET 1kb'],epi_labels=['CTCF'],x_ticks=['21MB','21.5MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_1kb_ctcf.png')

# RAD21

In [ ]:
outputpath = f'{rootpath}/gChIA/src/gchia/Figure/2/RAD21'

## 10KB

In [ ]:
cellline = 'GM12878'
# load data
# load data
start = 24000000
end = 26000000
resolution = 10000
epxname = '250316_GM12878_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_high_RAD21_ChIA-PET_log1p_true'
hic = np.load(ff'{rootpath}/gChIA/results/{epxname}/{cellline}/predictions/pred_matrix_chr10_{start}_{end}.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(ff'{rootpath}/gChIA/data/{cellline}/bigWig_files/{cellline}_RAD21.bw')
ctcf = bw.stats('chr10', start, end, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/RAD21_ChIA-PET/ENCFF416RNE.hic'
rpet = _read_hic_matrix(chiapet, 'chr10',start, end, resolution,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', start, end, resolution,log1p=False)
rhic = np.nan_to_num(rhic)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['Predicted 10kb','RAD21 ChIA-PET 10kb'],epi_labels=['RAD21'],x_ticks=['24MB','chr10','26MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_10kb_RAD21.png')


## 5KB

In [ ]:
cellline = 'GM12878'
# load data
# load data
start = 24000000
end = 26000000
resolution = 5000
epxname = '250316_GM12878_5000_NONE_ChIAPETMatrixPredictor_efeaturesq_high_RAD21_ChIA-PET_log1p_true'
hic = np.load(ff'{rootpath}/gChIA/results/{epxname}/{cellline}/predictions/pred_matrix_chr10_{start}_{end}.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(ff'{rootpath}/gChIA/data/{cellline}/bigWig_files/{cellline}_RAD21.bw')
ctcf = bw.stats('chr10', start, end, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/RAD21_ChIA-PET/ENCFF416RNE.hic'
rpet = _read_hic_matrix(chiapet, 'chr10',start, end, resolution,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', start, end, resolution,log1p=False)
rhic = np.nan_to_num(rhic)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['Predicted 5kb','RAD21 ChIA-PET 5kb'],epi_labels=['RAD21'],x_ticks=['24MB','chr10','26MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_5kb_RAD21.png')


## 1KB

In [ ]:
cellline = 'GM12878'
# load data
# load data
start = 24500000
end = 25000000
resolution = 1000
epxname = '250317_GM12878_1000_NONE_ChIAPETMatrixPredictor_efeaturesq_high_RAD21_ChIA-PET_log1p_true'
hic = np.load(ff'{rootpath}/gChIA/results/{epxname}/{cellline}/predictions/pred_matrix_chr10_{start}_{end}.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(ff'{rootpath}/gChIA/data/{cellline}/bigWig_files/{cellline}_RAD21.bw')
ctcf = bw.stats('chr10', start, end, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data
chiapet = f'{rootpath}/gChIA/data/GM12878/RAD21_ChIA-PET/ENCFF416RNE.hic'
rpet = _read_hic_matrix(chiapet, 'chr10',start, end, resolution,log1p=False)
rhic = f'{rootpath}/gChIA/data/GM12878/Hi-C/4DNFI1UEG1HD.hic'
rhic = _read_hic_matrix(rhic, 'chr10', start, end, resolution,log1p=False)
rhic = np.nan_to_num(rhic)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['Predicted 10kb','RAD21 ChIA-PET 10kb'],epi_labels=['RAD21'],x_ticks=['24.5MB','chr10','25MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_1kb_RAD21.png')


In [ ]:
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['rChIA 1kb','microC 1kb'],x_ticks=['21MB','21.5MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0,
    epi_labels=['CTCF','DNase-seq','H3K4me3','H3K27ac','H3K27me3','RAD21'],maxperc=97)

# 跨细胞系

## 10kb

In [ ]:
realhic = f'{rootpath}/gChIA/data/IMR90/4DNFIH7TH4MF.hic'
chiapet = f'{rootpath}/gChIA/data/IMR90/CTCF_ChIA-PET/ENCFF934OLI.hic'
# load data
hic = np.load(f'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878toIMR90/predictions/pred_matrix_chr10_21000000_23000000.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/IMR90/bigWig_files/IMR90_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 23000000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data

rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 23000000, 10000,log1p=False)

rhic = _read_hic_matrix(realhic, 'chr10', 21000000, 23000000, 10000,log1p=False)

visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['rChIA 10kb','ChIA-PET 10kb'],epi_labels=['CTCF'],x_ticks=['21MB','23MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05)

## 5kb

In [ ]:
hic = np.load(f'{rootpath}/gChIA/results/250304_5000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_True/GM12878toIMR90/predictions/pred_matrix_chr10_21000000_23000000.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/IMR90/bigWig_files/IMR90_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 23000000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data

rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 23000000, 5000,log1p=False)

rhic = _read_hic_matrix(realhic, 'chr10', 21000000, 23000000, 5000,log1p=False)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['rChIA 5kb','ChIA-PET 5kb'],epi_labels=['CTCF'],x_ticks=['21MB','23MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05)

In [ ]:
hic = np.load(f'{rootpath}/gChIA/results/250307_1000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878toIMR90/predictions/pred_matrix_chr10_21000000_21500000.npy')
hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
bw = pyBigWig.open(f'{rootpath}/gChIA/data/IMR90/bigWig_files/IMR90_ctcf.bw')
ctcf = bw.stats('chr10', 21000000, 23000000, type='mean', nBins=hic.shape[0])
ctcf = np.array(ctcf)

# Close the BigWig file
bw.close()

epi_list = [ctcf]
# real data

rpet = _read_hic_matrix(chiapet, 'chr10', 21000000, 21500000, 1000,log1p=False)

rhic = _read_hic_matrix(realhic, 'chr10', 21000000, 21500000, 1000,log1p=False)
visualize_multiple_HiC_with_epigenetics([hic,rpet], epi_list,hic_labels=['rChIA 1kb','ChIA-PET 1kb'],epi_labels=['CTCF'],x_ticks=['21MB','21.5MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05)

In [ ]:
cellline = 'GM12878'
# load data
# hic = np.load(f'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions/pred_matrix_chr10_21000000_23000000.npy')
# hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi



start = 15500000
end = start + 2000000
# real data
hic = np.load(ff'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions/pred_matrix_chr10_{start}_{end}.npy')
hic = np.expm1(hic)
bwGM = pyBigWig.open(f'{rootpath}/gChIA/data/GM12878/bigWig_files/GM12878_ctcf.bw')
ctcfGM = bwGM.stats('chr10', start, end, type='mean', nBins=hic.shape[0])
ctcfGM = np.array(ctcfGM)
epi_listGM = [ctcfGM]
bwIMR = pyBigWig.open(f'{rootpath}/gChIA/data/IMR90/bigWig_files/IMR90_ctcf.bw')
ctcfIMR = bwIMR.stats('chr10', start, end, type='mean', nBins=hic.shape[0])
ctcfIMR = np.array(ctcfIMR)
epi_listIMR = [ctcfIMR]
# Close the BigWig file
bw.close()
hicIM = np.load(ff'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878toIMR90/predictions/pred_matrix_chr10_{start}_{end}.npy')
hicIM = np.expm1(hicIM)
chiapet = f'{rootpath}/gChIA/data/GM12878/CTCF_ChIA-PET/ENCFF379AWZ.hic'
rpet = _read_hic_matrix(chiapet, 'chr10', start, end, 10000,log1p=False)
rhic = f'{rootpath}/gChIA/data/IMR90/CTCF_ChIA-PET/ENCFF934OLI.hic'
rhic = _read_hic_matrix(rhic, 'chr10', start, end, 10000,log1p=False)
rhic = np.nan_to_num(rhic)
outputpath = f'{rootpath}/gChIA/src/gchia/Figure/5'
visualize_multiple_HiC_with_epigenetics([rhic,hicIM], epi_listIMR,hic_labels=['IMR90','predicted'],epi_labels=['CTCF'],x_ticks=[f'{start/1000000}MB','chr10',f'{end/1000000}MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/IMR90_10kb_ctcf.png')
visualize_multiple_HiC_with_epigenetics([rpet,hic], epi_listGM,hic_labels=['GM12878','predicted'],epi_labels=['CTCF'],x_ticks=[f'{start/1000000}MB','chr10',f'{end/1000000}MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/GM12878_10kb_ctcf.png'
    )




# High-resulotion

In [ ]:
cellline = 'GM12878'
# load data
# hic = np.load(f'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions/pred_matrix_chr10_21000000_23000000.npy')
# hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
start = 21000000
end = start + 500000
# real data
prepath = f'{rootpath}/gChIA/results/250307_1000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions'
datapath = ff'{rootpath}/gChIA/data/{cellline}/bigWig_files'
pre = np.load(f'{prepath}/pred_matrix_chr10_{start}_{end}.npy')
pre = np.expm1(pre)
bwlist = ['ctcf']
bw = []
for i in bwlist:
    bw.append(pyBigWig.open(f'{datapath}/{cellline}_{i}.bw'))
epi_list = []
for i in range(len(bwlist)):
    bwf = bw[i].stats('chr10', start, end, type='mean', nBins=pre.shape[0])
    bwn = np.array(bwf)
    bw[i].close()
    epi_list.append(bwf)

# Close the BigWig file

realpath = f'{rootpath}/gChIA/data/GM12878/CTCF_ChIA-PET/ENCFF379AWZ.hic'
rpet = _read_hic_matrix(realpath, 'chr10', start, end, 1000,log1p=False)
outputpath = f'{rootpath}/gChIA/src/gchia/Figure/6'
os.path.exists(outputpath) or os.makedirs(outputpath)
visualize_multiple_HiC_with_epigenetics([rpet,pre], epi_list,hic_labels=['CTCF ChIA-PET 1kb','Predicted'],epi_labels=['CTCF'],x_ticks=[f'{start/1000000}MB','chr10',f'{end/1000000}MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/{cellline}_1kb_ctcf.png',
    maxperc=99)





In [ ]:
cellline = 'H1-hESC'
# load data
# hic = np.load(f'{rootpath}/gChIA/results/250303_10000_NONE_ChIAPETMatrixPredictor_efeaturesq_log1p_true/GM12878/predictions/pred_matrix_chr10_21000000_23000000.npy')
# hic = np.expm1(hic)
# main dioagonal
# hic[np.diag_indices(hic.shape[0])] = 0
# load epi
start = 21000000
end = start + 500000
# real data
prepath = f'{rootpath}/gChIA/results/250414_H1-hESC_1000_NONE_ChIAPETMatrixPredictor_efeaturesq_high_micro-c_log1p_true_hictype_hic_chiatype_hic_PREDICT_ONLY_hicres_5000/H1-hESCtoH1-hESC/predictions'
datapath = ff'{rootpath}/gChIA/data/{cellline}/bigWig_files'
pre = np.load(f'{prepath}/pred_matrix_chr10_{start}_{end}.npy')
pre = np.expm1(pre)
bwlist = ['ctcf','DNase-seq','H3K27ac']
bw = []
for i in bwlist:
    print(f'{datapath}/{cellline}_{i}.bw')
    bw.append(pyBigWig.open(f'{datapath}/{cellline}_{i}.bw'))
epi_list = []
for i in range(len(bwlist)):
    bwf = bw[i].stats('chr10', start, end, type='mean', nBins=pre.shape[0])
    bwn = np.array(bwf)
    bw[i].close()
    epi_list.append(bwf)

# Close the BigWig file

realpath = f'{rootpath}/gChIA/data/IMR90/Hi-C/4DNFIH7TH4MF.hic'
rpet = _read_hic_matrix(realpath, 'chr10', start, end, 1000,log1p=False)
outputpath = f'{rootpath}/gChIA/src/gchia/Figure/6'
os.path.exists(outputpath) or os.makedirs(outputpath)
visualize_multiple_HiC_with_epigenetics([rpet,pre],hic_labels=['Micro-C 1kb','Predicted'],x_ticks=[f'{start/1000000}MB','chr10',f'{end/1000000}MB'],colorbar=False,interval_between_hic=0,      
    interval_after_hic_block=0.05, 
    interval_between_epi=0.05,
    output=f'{outputpath}/{cellline}_1kb_micro.png',
    maxperc=97)



